<a href="https://colab.research.google.com/github/csansarricq/deep-learning-challenge/blob/main/AlphabetSoupCharity_Optimization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd
application_df = pd.read_csv("https://static.bc-edx.com/data/dl-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
#already removed EIN and NAME from the first run, maybe can remove status and special considerations? wasnt sure but removed ask_amt just in case
application_df = application_df.drop(columns = ['EIN', 'NAME', 'STATUS', 'SPECIAL_CONSIDERATIONS', 'ASK_AMT'])
application_df

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,0,1
1,T3,Independent,C2000,Preservation,Co-operative,1-9999,1
2,T5,CompanySponsored,C3000,ProductDev,Association,0,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,1
...,...,...,...,...,...,...,...
34294,T4,Independent,C1000,ProductDev,Association,0,0
34295,T4,CompanySponsored,C3000,ProductDev,Association,0,0
34296,T3,CompanySponsored,C2000,Preservation,Association,0,0
34297,T5,Independent,C3000,ProductDev,Association,0,1


In [4]:
# Determine the number of unique values in each column.
application_df.nunique()

,0
APPLICATION_TYPE,17
AFFILIATION,6
CLASSIFICATION,71
USE_CASE,5
ORGANIZATION,4
INCOME_AMT,9
IS_SUCCESSFUL,2


In [5]:
# Look at APPLICATION_TYPE value counts to identify and replace with "Other"
value_counts = application_df["APPLICATION_TYPE"].value_counts()
value_counts

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
T9,156


In [6]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = list(value_counts[value_counts < 500].index)
# Replace in dataframe
for cls in classifications_to_replace:
    application_df["APPLICATION_TYPE"] = application_df["APPLICATION_TYPE"].replace(cls,"Other")

# Check to make sure replacement was successful
application_df["APPLICATION_TYPE"].value_counts()

#this output is different than what has been shown as expected?

,count
APPLICATION_TYPE,
T3,27037
T4,1542
T6,1216
T5,1173
T19,1065
T8,737
T7,725
T10,528
Other,276


In [7]:
# Look at CLASSIFICATION value counts to identify and replace with "Other"
class_count = application_df["CLASSIFICATION"].value_counts()
class_count

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
...,...
C4120,1
C8210,1
C2561,1


In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1
#i am assuming from the class_count values
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].astype(str)
application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].str.strip().str.lower()
new_class = class_count.loc[class_count > 1]
new_class

,count
CLASSIFICATION,
C1000,17326
C2000,6074
C1200,4837
C3000,1918
C2100,1883
C7000,777
C1700,287
C4000,194
C5000,116


In [9]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
#  YOUR CODE GOES HERE
classifications_to_replace = class_count[(class_count < 600) | (class_count == 1)].index
# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")

# Check to make sure replacement was successful
application_df['CLASSIFICATION'].value_counts()

,count
CLASSIFICATION,
c1000,17326
c2000,6074
c1200,4837
c3000,1918
c2100,1883
...,...
c4120,1
c8210,1
c2561,1


In [10]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(application_df)
dummies.head()

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M
0,1,False,True,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
1,1,False,False,False,True,False,False,False,False,False,...,False,False,True,False,False,False,False,False,False,False
2,0,False,False,False,False,False,True,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,1,False,False,False,True,False,False,False,False,False,...,True,False,False,True,False,False,False,False,False,False
4,1,False,False,False,True,False,False,False,False,False,...,True,False,False,False,True,False,False,False,False,False


In [11]:
# Split our preprocessed data into our features and target arrays
y = dummies["IS_SUCCESSFUL"].values
X= dummies.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=75)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [13]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.

#added two more layers to try to run it more
#will try to change the activation type
#might switch the activation type order
#added more units
input_numbers = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

#Xpert Learning Assistant Chat+ helped me here
# First hidden layer
nn.add(tf.keras.layers.Dense(units=110, input_dim=input_numbers, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=90, activation='relu'))

#adding a Third hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation="sigmoid"))

#adding a Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation="sigmoid"))

#adding a Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation="sigmoid"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 110)                 │          11,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 90)                  │           9,990 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 80)                  │           7,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 70)                  │           5,670 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_4 (Dense)                      │ (None, 60)                  │           4,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_5 (Dense)                      │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,811 (151.61 KB)

 Trainable params: 38,811 (151.61 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [15]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.6725 - loss: 0.6100
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7284 - loss: 0.5581
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7314 - loss: 0.5510
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7324 - loss: 0.5479
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7328 - loss: 0.5489
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7315 - loss: 0.5472
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7324 - loss: 0.5445
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7311 - loss: 0.5475
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7387 - loss: 0.5387
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7325 - loss: 0.5429
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7425 - loss: 0.5326
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [16]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 5ms/step - accuracy: 0.7280 - loss: 0.5653
Loss: 0.56528639793396, Accuracy: 0.7280466556549072


In [17]:
#changed the activation just to see if it would run differently

input_numbers = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

#Xpert Learning Assistant Chat+ helped me here
nn.add(tf.keras.layers.Dense(units=110, input_dim=input_numbers, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=90, activation='relu'))

#adding a Third hidden layer
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))

#adding a Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=70, activation='relu'))

#adding a Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=60, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                      │ (None, 110)                 │          11,550 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_7 (Dense)                      │ (None, 90)                  │           9,990 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_8 (Dense)                      │ (None, 80)                  │           7,280 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ (None, 70)                  │           5,670 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ (None, 60)                  │           4,260 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_11 (Dense)                     │ (None, 1)                   │              61 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 38,811 (151.61 KB)

 Trainable params: 38,811 (151.61 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 6s 3ms/step - accuracy: 0.7034 - loss: 0.5904
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7236 - loss: 0.5567
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7350 - loss: 0.5476
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7312 - loss: 0.5496
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7256 - loss: 0.5560
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.7375 - loss: 0.5435
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7352 - loss: 0.5410
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7344 - loss: 0.5428
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step - accuracy: 0.7364 - loss: 0.5393
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7371 - loss: 0.5442
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 3s 3ms/step - accuracy: 0.7345 - loss: 0.5441
Epoch 12/100
804/804 ━━━━━━━━━━━━━━━━━━━━

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - 4ms/step - accuracy: 0.7363 - loss: 0.6359
Loss: 0.635895311832428, Accuracy: 0.7363265156745911


In [21]:
#changed the unit numbers to see if things would look different
input_numbers = len(X_train_scaled[0])

nn = tf.keras.models.Sequential()

#Xpert Learning Assistant Chat+ helped me here
nn.add(tf.keras.layers.Dense(units=500, input_dim=input_numbers, activation='relu'))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=400, activation='relu'))

#adding a Third hidden layer
nn.add(tf.keras.layers.Dense(units=300, activation='relu'))

#adding a Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=200, activation='sigmoid'))

#adding a Fifth hidden layer
nn.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))
# Check the structure of the model
nn.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_12 (Dense)                     │ (None, 500)                 │          52,500 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_13 (Dense)                     │ (None, 400)                 │         200,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 300)                 │         120,300 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 200)                 │          60,200 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_16 (Dense)                     │ (None, 100)                 │          20,100 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_17 (Dense)                     │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 453,601 (1.73 MB)

 Trainable params: 453,601 (1.73 MB)

 Non-trainable params: 0 (0.00 B)

In [22]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7102 - loss: 0.5877
Epoch 2/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7275 - loss: 0.5589
Epoch 3/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - accuracy: 0.7319 - loss: 0.5502
Epoch 4/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 10s 10ms/step - accuracy: 0.7305 - loss: 0.5530
Epoch 5/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 9s 9ms/step - accuracy: 0.7280 - loss: 0.5518
Epoch 6/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - accuracy: 0.7234 - loss: 0.5556
Epoch 7/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 12s 13ms/step - accuracy: 0.7266 - loss: 0.5512
Epoch 8/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.7297 - loss: 0.5489
Epoch 9/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7308 - loss: 0.5489
Epoch 10/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 10s 9ms/step - accuracy: 0.7309 - loss: 0.5457
Epoch 11/100
804/804 ━━━━━━━━━━━━━━━━━━━━ 11s 9ms/step - accuracy: 0.7324 - loss: 0.5452
Epoch 12/100
804/804 ━━━━━━━━━

In [24]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 2s - 8ms/step - accuracy: 0.7335 - loss: 0.5608
Loss: 0.56078040599823, Accuracy: 0.7335277199745178
